## BPR: Bayesian Personalized Ranking from Implicit Feedback

Ref: 
* https://arxiv.org/pdf/1205.2618
* https://medium.com/radon-dev/implicit-bayesian-personalized-ranking-in-tensorflow-b4dfa733c478

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import scipy.sparse as sp

from tqdm import tqdm

In [2]:
#---------------------------
# LOAD AND PREPARE THE DATA
#---------------------------

# Load the dataframe from a tab separated file.
df = pd.read_csv('data/movielens/ml-20m/ratings.csv', sep=',')
    
# Add column names
df = df.drop(df.columns[3], axis=1)


In [3]:
df.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [4]:
df_movie = pd.read_csv('data/movielens/ml-20m/movies.csv')

In [5]:
df_movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:

# Drop any rows with missing values
df = df.dropna()

# Drop any rows with 0 rating
df = df.loc[df.rating != 0]

# Convert movies into numerical IDs
df['user_id'] = df['userId'].astype("category").cat.codes
df['movie_id'] = df['movieId'].astype("category").cat.codes

# Create a lookup frame so we can get the movie
# names back in readable form later.
item_lookup = df[['movie_id', 'movieId']].drop_duplicates()
item_lookup['movie_id'] = item_lookup.movie_id.astype(str)

# We drop our old user and item columns
df = df.drop(['userId', 'movieId'], axis=1)

# Drop any rows with 0 rating
df = df.loc[df.rating != 0]

# Create lists of all users, movies and ratings
users = list(np.sort(df.user_id.unique()))
movies = list(np.sort(df.movie_id.unique()))
ratings = list(df.rating)
print(f"#users: {len(users):,}, #items: {len(movies):,}")

# Get the rows and columns for our new matrix
rows = df.user_id.astype(float)
cols = df.movie_id.astype(float)

# Contruct a sparse matrix for our users and items containing number of ratings
data_sparse = sp.csr_matrix((ratings, (rows, cols)), shape=(len(users), len(movies)))

# Get the values of our matrix as a list of user ids
# and item ids. Note that our litsts have the same length
# as each user id repeats one time for each rated movie.
uids, iids = data_sparse.nonzero()

#users: 138,493, #items: 26,744


In [7]:
#-------------
# HYPERPARAMS
#-------------

epochs = 50
batches = 30
num_factors = 64 # Number of latent features

# Independent lambda regularization values 
# for user, items and bias.
lambda_user = 0.0000001
lambda_item = 0.0000001
lambda_bias = 0.0000001

# Our learning rate 
lr = 0.005

# How many (u,i,j) triplets we sample for each batch
samples = 15000

In [8]:
#-------------------------
# TENSORFLOW GRAPH
#-------------------------

# Set up our Tensorflow graph
graph = tf.Graph()

def init_variable(size, dim, name=None):
    '''
    Helper function to initialize a new variable with
    uniform random values.
    '''
    std = np.sqrt(2 / dim)
    return tf.Variable(tf.random_uniform([size, dim], -std, std), name=name)


def embed(inputs, size, dim, name=None):
    '''
    Helper function to get a Tensorflow variable and create
    an embedding lookup to map our user and item
    indices to vectors.
    '''
    emb = init_variable(size, dim, name)
    return tf.nn.embedding_lookup(emb, inputs)


def get_variable(graph, session, name):
    '''
    Helper function to get the value of a
    Tensorflow variable by name.
    '''
    v = graph.get_operation_by_name(name)
    v = v.values()[0]
    v = v.eval(session=session)
    return v

In [9]:
with graph.as_default():
    '''
    Loss function: 
    -SUM ln σ(xui - xuj) + λ(w1)**2 + λ(w2)**2 + λ(w3)**2 ...
    ln = the natural log
    σ(xuij) = the sigmoid function of xuij.
    λ = lambda regularization value.
    ||W||**2 = the squared L2 norm of our model parameters.
    
    '''

    # Input into our model, in this case our user (u),
    # known item (i) an unknown item (i) triplets.
    u = tf.placeholder(tf.int32, shape=(None, 1))
    i = tf.placeholder(tf.int32, shape=(None, 1))
    j = tf.placeholder(tf.int32, shape=(None, 1))

    # User feature embedding
    u_factors = embed(u, len(users), num_factors, 'user_factors') # U matrix

    # Known and unknown item embeddings
    item_factors = init_variable(len(movies), num_factors, "item_factors") # V matrix
    i_factors = tf.nn.embedding_lookup(item_factors, i)
    j_factors = tf.nn.embedding_lookup(item_factors, j)

    # i and j bias embeddings.
    item_bias = init_variable(len(movies), 1, "item_bias")
    i_bias = tf.nn.embedding_lookup(item_bias, i)
    i_bias = tf.reshape(i_bias, [-1, 1])
    j_bias = tf.nn.embedding_lookup(item_bias, j)
    j_bias = tf.reshape(j_bias, [-1, 1])

    # Calculate the dot product + bias for known and unknown
    # item to get xui and xuj.
    xui = i_bias + tf.reduce_sum(u_factors * i_factors, axis=2)
    xuj = j_bias + tf.reduce_sum(u_factors * j_factors, axis=2)

    # We calculate xuij.
    xuij = xui - xuj

    # Calculate the mean AUC (area under curve).
    # if xuij is greater than 0, that means that 
    # xui is greater than xuj (and thats what we want).
    u_auc = tf.reduce_mean(tf.to_float(xuij > 0))

    # Output the AUC value to tensorboard for monitoring.
    tf.summary.scalar('auc', u_auc)

    # Calculate the squared L2 norm ||W||**2 multiplied by λ.
    l2_norm = tf.add_n([
        lambda_user * tf.reduce_sum(tf.multiply(u_factors, u_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(i_factors, i_factors)),
        lambda_item * tf.reduce_sum(tf.multiply(j_factors, j_factors)),
        lambda_bias * tf.reduce_sum(tf.multiply(i_bias, i_bias)),
        lambda_bias * tf.reduce_sum(tf.multiply(j_bias, j_bias))
        ])

    # Calculate the loss as ||W||**2 - ln σ(Xuij)
    #loss = l2_norm - tf.reduce_mean(tf.log(tf.sigmoid(xuij)))
    loss = -tf.reduce_mean(tf.log(tf.sigmoid(xuij))) + l2_norm
    
    # Train using the Adam optimizer to minimize 
    # our loss function.
    opt = tf.train.AdamOptimizer(learning_rate=lr)
    step = opt.minimize(loss)

    # Initialize all tensorflow variables.
    init = tf.global_variables_initializer()

W0724 09:05:41.277662 4481074624 deprecation.py:323] From <ipython-input-9-6d216b63fecb>:44: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [11]:
#------------------
# GRAPH EXECUTION
#------------------

# Run the session. 
session = tf.Session(config=None, graph=graph)
session.run(init)

# This has noting to do with tensorflow but gives
# us a nice progress bar for the training.
progress = tqdm(total=batches*epochs)

for _ in range(epochs):
    for _ in range(batches):

        # We want to sample one known and one unknown 
        # item for each user. 

        # First we sample 15000 uniform indices.
        idx = np.random.randint(low=0, high=len(uids), size=samples)

        # We then grab the users matching those indices.
        batch_u = uids[idx].reshape(-1, 1)

        # Then the known items for those users.
        batch_i = iids[idx].reshape(-1, 1)

        # Lastly we randomly sample one unknown item for each user.
        batch_j = np.random.randint(
                low=0, high=len(movies), size=(samples, 1), dtype='int32')

        # Feed our users, known and unknown items to
        # our tensorflow graph. 
        feed_dict = { u: batch_u, i: batch_i, j: batch_j }

        # We run the session.
        _, l, auc = session.run([step, loss, u_auc], feed_dict)

    progress.update(batches)
    progress.set_description('Loss: %.3f | AUC: %.3f' % (l, auc))

progress.close()


  2%|▏         | 30/1500 [00:02<01:59, 12.29it/s]
Loss: 0.764 | AUC: 0.559:   2%|▏         | 30/1500 [00:02<01:59, 12.29it/s]
Loss: 0.764 | AUC: 0.559:   4%|▍         | 60/1500 [00:04<01:58, 12.20it/s]
Loss: 0.689 | AUC: 0.609:   4%|▍         | 60/1500 [00:04<01:58, 12.20it/s]
Loss: 0.689 | AUC: 0.609:   6%|▌         | 90/1500 [00:07<01:55, 12.16it/s]
Loss: 0.621 | AUC: 0.660:   6%|▌         | 90/1500 [00:07<01:55, 12.16it/s]
Loss: 0.621 | AUC: 0.660:   8%|▊         | 120/1500 [00:10<01:54, 12.00it/s]
Loss: 0.554 | AUC: 0.712:   8%|▊         | 120/1500 [00:10<01:54, 12.00it/s]
Loss: 0.554 | AUC: 0.712:  10%|█         | 150/1500 [00:12<01:54, 11.75it/s]
Loss: 0.470 | AUC: 0.776:  10%|█         | 150/1500 [00:12<01:54, 11.75it/s]
Loss: 0.470 | AUC: 0.776:  12%|█▏        | 180/1500 [00:15<01:54, 11.55it/s]
Loss: 0.356 | AUC: 0.866:  12%|█▏        | 180/1500 [00:15<01:54, 11.55it/s]
Loss: 0.356 | AUC: 0.866:  14%|█▍        | 210/1500 [00:17<01:50, 11.68it/s]
Loss: 0.257 | AUC: 0.920:  14%